In [22]:
import pandas as pd

df = pd.read_parquet('/Users/Macington/Documents/Projects/Project Gambit/Games/Storage/total_ply=47/data.parquet')

df

,game_id,pgn,ply,board_sum,centipawn_evaluation,final_centipawn_value,board_sum_in_context
0,4.257591e+20,"[Event ""Mar del Plata""]\n[Site ""Mar del Plata""...",0,18446462598732906495,None,None,1.844646e+19
1,4.257591e+20,"[Event ""Mar del Plata""]\n[Site ""Mar del Plata""...",1,18446462598800014335,None,None,3.689293e+19
2,4.257591e+20,"[Event ""Mar del Plata""]\n[Site ""Mar del Plata""...",2,18441976591358688255,None,None,5.533490e+19
3,4.257591e+20,"[Event ""Mar del Plata""]\n[Site ""Mar del Plata""...",3,18441976591360785343,None,None,7.377688e+19
4,4.257591e+20,"[Event ""Mar del Plata""]\n[Site ""Mar del Plata""...",4,18439724825906838463,None,None,9.221660e+19
...,...,...,...,...,...,...,...
29699,4.309053e+20,"[Event ""Las Palmas""]\n[Site ""Las Palmas""]\n[Da...",42,5256615563724092224,None,None,4.104553e+20
29700,4.309053e+20,"[Event ""Las Palmas""]\n[Site ""Las Palmas""]\n[Da...",43,5256615563724222784,None,None,4.157119e+20
29701,4.309053e+20,"[Event ""Las Palmas""]\n[Site ""Las Palmas""]\n[Da...",44,5256615559429255488,None,None,4.209685e+20
29702,4.309053e+20,"[Event ""Las Palmas""]\n[Site ""Las Palmas""]\n[Da...",45,5256615559429251392,None,None,4.262251e+20


In [ ]:
import pyarrow.dataset as ds
import pyarrow.parquet as pq
import pyarrow as pa

storage_directory = "/Users/Macington/Documents/Projects/Project Gambit/Games/Storage"

# Create a dataset object
dataset = ds.dataset(storage_directory, format="parquet")

# Loop through fragments (partitions) in the dataset
for fragment in dataset.get_fragments():
    # Read a fragment into a Table
    table = fragment.to_table()
    # Convert to pandas DataFrame
    df = table.to_pandas()

    # Calculate the cumulative sum for each game
    df['board_sum_in_context'] = df.groupby('game_id')['board_sum'].transform(lambda x: (x + df.loc[x.index, 'ply']).cumsum())

    # Convert back to pyarrow Table
    updated_table = pa.Table.from_pandas(df)

    # Determine the path to write the updated fragment
    output_path = fragment.path

    # Write the updated Table back to the fragment's location
    pq.write_table(updated_table, output_path)

print("Updated all partitions.")


In [ ]:
import pandas as pd
from pathlib import Path
import os

def update_partitions(storage_directory):
    # Iterate through all the subdirectories (partitions) in the storage directory
    for partition_dir in os.listdir(storage_directory):
        partition_path = Path(storage_directory, partition_dir)
        if not partition_path.is_dir():
            continue
        
        # Iterate through all the parquet files in the partition directory
        for file_path in partition_path.glob("*.parquet"):
            print(f"Processing file: {file_path}")
            
            # Read the parquet file
            df = pd.read_parquet(file_path)

            df.drop(columns=['centipawn_diff', 'final_centipawn_value_x', 'final_centipawn_value_y'], errors='ignore', inplace=True)

            # Remove duplicates based on game_id and ply, keeping the same sort order
            df.drop_duplicates(subset=['game_id', 'ply'], inplace=True)
            
            # Get the final centipawn value for each game_id
            final_centipawn_values = df.groupby('game_id')['centipawn_evaluation'].last()
            final_centipawn_values_dict = final_centipawn_values.to_dict()

            # Update the final_centipawn_value column using the mapping
            df['final_centipawn_value'] = df['game_id'].map(final_centipawn_values_dict)

            # Reindex the DataFrame
            df.reset_index(drop=True, inplace=True)

            # Save the updated DataFrame back to the parquet file
            df.to_parquet(file_path)

            print(f"File {file_path} updated.")

    print("All partitions processed.")

if __name__ == "__main__":
    storage_directory = "/Users/Macington/Documents/Projects/Project Gambit/Games/Storage"
    update_partitions(storage_directory)

In [21]:
from pathlib import Path
import pyarrow.dataset as ds
import pyarrow.parquet as pq
import pyarrow as pa
import pandas as pd

def reset_values(storage_directory, start_ply, end_ply):
    # Create a dataset object
    dataset = ds.dataset(storage_directory, format="parquet")

    # Loop through fragments (partitions) in the dataset
    for fragment in dataset.get_fragments():
        # Extract total_ply from the fragment's path
        fragment_path = Path(fragment.path)
        total_ply_str = fragment_path.parent.name.split('=')[-1]
        total_ply = int(total_ply_str)

        if start_ply <= total_ply <= end_ply:
            # Read a fragment into a Table
            table = fragment.to_table()
            # Convert to pandas DataFrame
            df = table.to_pandas()

            # Set centipawn_evaluation and final_centipawn_value to NaN
            df['centipawn_evaluation'] = None
            df['final_centipawn_value'] = None

            # Convert back to pyarrow Table
            updated_table = pa.Table.from_pandas(df)

            # Write the updated Table back to the fragment's location
            pq.write_table(updated_table, fragment.path)

    print(f"Updated centipawn_evaluation and final_centipawn_value to null for total_ply={start_ply} to {end_ply}.")

def main():
    storage_directory = "/Users/Macington/Documents/Projects/Project Gambit/Games/Storage"
    start_ply = 47
    end_ply = 228

    # Reset centipawn_evaluation and final_centipawn_value to null
    reset_values(storage_directory, start_ply, end_ply)

if __name__ == "__main__":
    main()


Updated centipawn_evaluation and final_centipawn_value to null for total_ply=47 to 228.


In [ ]:
import chess.pgn
import pandas as pd
from pathlib import Path
from io import StringIO
import gc

def load_previously_evaluated_sequences(storage_directory, start_ply, end_ply):
    evaluated_sequences = {}
    for total_ply in range(start_ply, end_ply + 1):
        partition_path = Path(storage_directory, f"total_ply={total_ply}", "data.parquet")
        if not partition_path.exists():
            continue

        df = pd.read_parquet(partition_path)
        for pgn, evaluation in zip(df['pgn'], df['centipawn_evaluation']):
            moves = [str(move) for move in chess.pgn.read_game(StringIO(pgn)).mainline_moves()]
            for i in range(1, len(moves) + 1):
                evaluated_sequences[' '.join(moves[:i])] = evaluation

    return evaluated_sequences

def match_and_join_sequences(storage_directory, storage_test_directory, start_ply, end_ply, evaluated_sequences):
    print("Starting matching and joining sequences...")
    for total_ply in range(start_ply, end_ply + 1):
        partition_path = Path(storage_directory, f"total_ply={total_ply}", "data.parquet")
        if not partition_path.exists():
            continue

        df = pd.read_parquet(partition_path)
        evaluations = []

        for game_id, pgn in df[['game_id', 'pgn']].itertuples(index=False):
            moves = [str(move) for move in chess.pgn.read_game(StringIO(pgn)).mainline_moves()]
            for i in range(1, len(moves) + 1):
                move_sequence = ' '.join(moves[:i])
                evaluation = evaluated_sequences.get(move_sequence, None)
                if evaluation is not None:
                    evaluations.append((game_id, i, evaluation))

        # Create a DataFrame with the evaluations
        eval_df = pd.DataFrame(evaluations, columns=['game_id', 'ply', 'centipawn_evaluation'])
        df = df.merge(eval_df, on=['game_id', 'ply'], how='left')
        new_partition_dir = Path(storage_test_directory, f"total_ply={total_ply}")
        new_partition_dir.mkdir(exist_ok=True)
        new_partition_path = new_partition_dir / "data.parquet"
        df.to_parquet(new_partition_path)

        print(f"Saved partition for total_ply={total_ply}...")
        gc.collect()  # Call garbage collection inside the loop

    print("Processing completed!")

def main():
    print("Starting the main function...")
    storage_directory = "/Users/Macington/Documents/Projects/Project Scotch/Games/Storage"
    storage_test_directory = "/Users/Macington/Documents/Projects/Project Gambit/Games/Storage"
    start_ply = 47
    end_ply = 228

    # Load previously evaluated sequences
    print("Calling load_previously_evaluated_sequences function...")
    evaluated_sequences = load_previously_evaluated_sequences(storage_test_directory, start_ply - 1, end_ply - 1) # Adjust start and end plies if needed

    # Match and join sequences
    print("Calling match_and_join_sequences function...")
    match_and_join_sequences(storage_directory, storage_test_directory, start_ply, end_ply, evaluated_sequences)

    print("Main function execution completed.")

if __name__ == "__main__":
    main()

# 47 to 228
